# Мониторинг модели с помощью Prometheus и Grafana

В этом проекте мы потренеруемся в мониторинге моделей. Для этой задачи возьмем датасет вино и обучим на нем несколько моделей с последующим мониторингом.

Загрузим необходимые библиотеки и данные.

In [68]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import joblib
import pandas as pd
import numpy as np

In [69]:
dataset = pd.read_csv('winequality-red.zip', sep=';')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [70]:
dataset.describe()

fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000              6.000000     0.990070   
25%       0.070000             7.000000             22.000000     0.995600   
50%       0.079000            14.000000             38.000000     0.996750   
75%       0.090000            21.000000             62.000000     0.997835   
max       0.611000            72.000000            289.000000     1.003690   

                pH    sulphates      alcohol      quality  
count  1599.000000  1599.000000  1599.000000  1599.000000  
mean      3.311113     0.658149    10.422983     5.636023  
std       0.154386     0.169507     1.065668     0.807569  
min       2.740000     0.330000     8.400000     3.000000  
25%       3.210000     0.550000     9.500000     5.000000  
50%       3.310000     0.620000    10.200000     6.000000  
75%       3.400000     0.730000    11.100000     6.000000  
max       4.010000     2.000000    14.900000     8.000000

In [71]:
dataset.quality.value_counts().sort_index()

quality
3     10
4     53
5    681
6    638
7    199
8     18
Name: count, dtype: int64

Что ж, пропусков нет, все признаки числовые. Для упрощения переведем задачу в бинарный вид "хорошее и плохое" вино. "Хорошим" будет вино с качеством 7-8, остальное запишем в "плохое". При обучении нужно будет учесть веса классов, так как классы не сбалансированны. Также стандартизируем остальные признаки.

In [72]:
X = dataset.drop('quality', axis = 1)
y = dataset['quality'].apply(lambda x: 1 if x > 6 else 0)

In [73]:
n_samples = y.shape[0]
n_class_0, n_class_1 = np.bincount(y)
weight_for_0 = (1 / n_class_0) * (n_samples / 2.0)
weight_for_1 = (1 / n_class_1) * (n_samples / 2.0)
class_weights = {0: weight_for_0, 1: weight_for_1}
print("Веса классов:", class_weights)

Веса классов: {0: 0.5785094066570188, 1: 3.684331797235023}


In [74]:
# Стандартизация признаков
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на тренировочную и тестовую выборки
train_X, test_X, train_y, test_y = train_test_split(X, y.to_numpy(), test_size=0.2, random_state=42)

## Модель LightGBM

In [75]:
def streaming_reading(X_train, y_train, batch_size=5000):
    X, y = [], []
    current_line = 0
    train_data, train_label = shuffle(X_train, y_train, random_state=0)
    train_data = train_data
    for row, target in zip(train_data, train_label):
        X.append(row)
        y.append(target)
        current_line += 1
        if current_line >= batch_size:
            X, y = np.array(X), np.array(y)
            yield X, y
            X, y = [], []
            current_line = 0
    if X:
        yield np.array(X), np.array(y)

In [76]:
import lightgbm as lgb

def train_lightgbm_incremental(X, y, class_weights):
    gbm = None
    params = {
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'],
        'learning_rate': 0.05,
        'max_bin': 255,
        'verbosity': -1,
        'scale_pos_weight': class_weights[1] / class_weights[0]  # Вес для положительного класса
    }
    streaming_train_iterators = streaming_reading(X, y, batch_size=500)  # Уменьшен батч для примера
    
    for i, data in enumerate(streaming_train_iterators):
        X_batch, y_batch = data
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, test_size=0.1, random_state=0)
        lgb_train = lgb.Dataset(X_train, y_train, weight=[class_weights[label] for label in y_train])
        lgb_eval = lgb.Dataset(X_test, y_test, weight=[class_weights[label] for label in y_test], reference=lgb_train)
        gbm = lgb.train(params, 
                        lgb_train, 
                        num_boost_round=1000, 
                        valid_sets=[lgb_eval],
                        init_model=gbm, 
                        callbacks=[lgb.early_stopping(stopping_rounds=10)],
                        keep_training_booster=True)
        
        print(f"{i} time")
        score_train = dict([(score[1], score[2]) for score in gbm.eval_train()])
        print(f"Logloss: {score_train['binary_logloss']:.4f}, AUC: {score_train['auc']:.4f}")
    
    return gbm



In [77]:
# Обучение
lgb_model = train_lightgbm_incremental(train_X, train_y, class_weights)
pred_y_lgb = lgb_model.predict(test_X)
pred_classes_lgb = np.where(pred_y_lgb > 0.5, 1, 0)

joblib.dump(lgb_model, 'lgb_model.pkl')
lgb_model = joblib.load('lgb_model.pkl')

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's binary_logloss: 0.597806	valid_0's auc: 0.92
0 time
Logloss: 0.4410, AUC: 0.9607


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	valid_0's binary_logloss: 0.541072	valid_0's auc: 0.80506
1 time
Logloss: 0.3938, AUC: 0.9420
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	valid_0's binary_logloss: 0.363332	valid_0's auc: 0.973913
2 time
Logloss: 0.4100, AUC: 0.8977


## Модель CatBoost

В CatBoost нет прямого аналога keep_training_booster, но дообучение реализуется через параметр init_model, который позволяет передать ранее обученную модель для продолжения обучения.

In [78]:
from catboost import CatBoostClassifier, Pool
import numpy as np

def train_catboost_incremental(X, y, class_weights):
    model = None
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'learning_rate': 0.05,
        'iterations': 1000,
        'verbose': 0,
        'class_weights': [class_weights[0], class_weights[1]]  # Вес для класса 0 и класса 1
    }
    streaming_train_iterators = streaming_reading(X, y, batch_size=50)
    
    for i, data in enumerate(streaming_train_iterators):
        X_batch, y_batch = data
        X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, test_size=0.1, random_state=0)
        
        # Создаём Pool для тренировочных и тестовых данных
        train_pool = Pool(X_train, y_train)
        eval_pool = Pool(X_test, y_test)
        
        if model is None:
            model = CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=eval_pool, early_stopping_rounds=10)
        else:
            model.fit(train_pool, eval_set=eval_pool, init_model=model, early_stopping_rounds=10)
        
        print(f"{i} time")
        # Оцениваем метрики с использованием Pool
        metrics = model.eval_metrics(eval_pool, metrics=['AUC'])
        auc = metrics['AUC'][-1]
        print(f"AUC: {auc:.4f}")
    
    return model

In [79]:
# Обучение
cat_model = train_catboost_incremental(train_X, train_y, class_weights)
pred_y_cat = cat_model.predict_proba(test_X)[:, 1]
pred_classes_cat = np.where(pred_y_cat > 0.5, 1, 0)

joblib.dump(cat_model, 'cat_model.pkl')
cat_model = joblib.load('cat_model.pkl')

0 time
AUC: 1.0000
1 time
AUC: 0.8333
2 time
AUC: 0.0000
3 time
AUC: 0.0000
4 time
AUC: 0.5000
5 time
AUC: 1.0000
6 time
AUC: 0.0000
7 time
AUC: 1.0000
8 time
AUC: 1.0000
9 time
AUC: 0.0000
10 time
AUC: 0.0000
11 time
AUC: 1.0000
12 time
AUC: 0.0000
13 time
AUC: 1.0000
14 time
AUC: 0.0000
15 time
AUC: 0.7500
16 time
AUC: 0.0000
17 time
AUC: 1.0000
18 time
AUC: 1.0000
19 time
AUC: 0.7500
20 time
AUC: 1.0000
21 time
AUC: 0.0000
22 time
AUC: 1.0000
23 time
AUC: 1.0000
24 time
AUC: 0.0000
25 time
AUC: 0.0000


## Логистическая регрессия

В SGDClassifier из scikit-learn дообучение реализуется через метод partial_fit, который позволяет обновлять модель на новых данных.
Параметр warm_start=True в SGDClassifier частично выполняет роль, схожую с keep_training_booster. 
Он указывает, что модель должна продолжить обучение с текущими весами, а не инициализировать их заново.

In [86]:
from sklearn.linear_model import SGDClassifier

def train_logistic_regression_incremental(X, y, class_weights):
    model = SGDClassifier(loss='log_loss', 
                          warm_start=True, 
                          class_weight=class_weights, 
                          max_iter=1000, 
                          tol=1e-3, 
                          random_state=42)
    streaming_train_iterators = streaming_reading(X, y, batch_size=50)
    
    for i, data in enumerate(streaming_train_iterators):
        X_batch, y_batch = data
        model.partial_fit(X_batch, y_batch, classes=[0, 1])
        
        print(f"{i} time")
        # Оценка на последнем батче (примерно)
        score = model.score(X_batch, y_batch)
        print(f"Accuracy: {score:.4f}")

    return model

In [87]:
# Обучение
log_model = train_logistic_regression_incremental(train_X, train_y, class_weights)
pred_y_log = log_model.predict_proba(test_X)[:, 1]
pred_classes_log = np.where(pred_y_log > 0.5, 1, 0)

joblib.dump(log_model, 'log_model.pkl')
log_model = joblib.load('log_model.pkl')

0 time
Accuracy: 0.7000
1 time
Accuracy: 0.7600
2 time
Accuracy: 0.7000
3 time
Accuracy: 0.7600
4 time
Accuracy: 0.7800
5 time
Accuracy: 0.8800
6 time
Accuracy: 0.8400
7 time
Accuracy: 0.9000
8 time
Accuracy: 0.8200
9 time
Accuracy: 0.7800
10 time
Accuracy: 0.8000
11 time
Accuracy: 0.7800
12 time
Accuracy: 0.7400
13 time
Accuracy: 0.7200
14 time
Accuracy: 0.7200
15 time
Accuracy: 0.9000
16 time
Accuracy: 0.7000
17 time
Accuracy: 0.8200
18 time
Accuracy: 0.7400
19 time
Accuracy: 0.8200
20 time
Accuracy: 0.7000
21 time
Accuracy: 0.7800
22 time
Accuracy: 0.8600
23 time
Accuracy: 0.8800
24 time
Accuracy: 0.6000
25 time
Accuracy: 0.6552


## Оценка моделей

In [88]:
for name, pred_classes in [("LightGBM", pred_classes_lgb), ("CatBoost", pred_classes_cat), ("Logistic", pred_classes_log)]:
    f1 = f1_score(test_y, pred_classes)
    precision = precision_score(test_y, pred_classes)
    recall = recall_score(test_y, pred_classes)
    auc = roc_auc_score(test_y, pred_classes)
    print(f"{name}:")
    print(f"F1-score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"AUC: {auc:.4f}")
    print("----------------------------------------")

LightGBM:
F1-score: 0.5031
Precision: 0.3571
Recall: 0.8511
AUC: 0.7937
----------------------------------------
CatBoost:
F1-score: 0.6429
Precision: 0.5538
Recall: 0.7660
AUC: 0.8299
----------------------------------------
Logistic:
F1-score: 0.4545
Precision: 0.3271
Recall: 0.7447
AUC: 0.7405
----------------------------------------


## Вывод

Видно, что лучше всего себя показала модель CatBoost (F1-score, AUC). У модели лучший баланс между Precision и Recall, а также она лучше всех справляется с различием классов.

Приступим к мониторингу CatBoost, как лучшей модели. Сделаем эмитацию поступления данных в реальном времени на основе нашего датасета, поделив его на батчи по 50шт. Программа по созданию метрик monitor.py, прошем в Prometheus соответствующие строки по настройке подключения нашего "сервиса" и отобразим метрики в Grafana.

    - job_name: 'model_metrics'
        static_configs:
          - targets: ['localhost:8000']